Install COBREXA if not installed yet, and load it

In [1]:
import Pkg
Pkg.add("COBREXA")
using COBREXA

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


Let's download and open the big model again

In [2]:
import Downloads
Downloads.download("http://bigg.ucsd.edu/static/models/iJO1366.json", "ecoli.json")
model = load_model(StandardModel, "ecoli.json");

Main reference is this:
https://lcsb-biocore.github.io/COBREXA.jl/stable/examples/07_gene_deletion/

Each reaction has a gene association, which dictates the gene products that
need to be available so that the reaction can "run".

In [3]:
genes(model)

gene_rules_dnf = reaction_gene_association(model, "PFK")

2-element Vector{Vector{String}}:
 ["b1723"]
 ["b3916"]

The result is in DNF for (computational) simplicity; the rules can be
converted e.g. to Strings which are more suitable for reading:

In [4]:
COBREXA._unparse_grr(String, gene_rules_dnf)

"(b1723) || (b3916)"

We might knock out genes by running through the reactions and evaluating DNF.
The knockout is available as a modification for simplicity:

In [5]:
gene_name(model, "b0720")

"gltA"

We need a solver

In [6]:
Pkg.add("GLPK")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


Run the knockout (implemented as an analysis modification for convenience)

In [7]:
using GLPK
sol = flux_balance_analysis_dict(
    model,
    GLPK.Optimizer,
    modifications = [knockout("b0720")],
)
sol["BIOMASS_Ec_iJO1366_core_53p95M"]

4.7967818666792276e-32

...the model is still feasible (so it can "sustain itself"), but growth is
basically zero.

We can screen through all genes. One could simply write something like:

In [8]:
[
    flux_balance_analysis_dict(model, GLPK.Optimizer, modifications = [knockout(g)]) for
    g in genes(model)[1:10]
]

10-element Vector{Dict{String, Float64}}:
 Dict("Zn2tex" => 0.00033498878813989854, "GUI1" => 0.0, "DXYLK" => 0.0, "CBL1tonex" => 0.0, "FE3DCITtonex" => 0.0, "FACOAL180t2pp" => 0.0, "METSOXR1" => 0.0, "LIPOtex" => 0.0, "NTD11" => 0.0, "GLUNpp" => 0.0…)
 Dict("Zn2tex" => 0.00033498878813989854, "GUI1" => 0.0, "DXYLK" => 0.0, "CBL1tonex" => 0.0, "FE3DCITtonex" => 0.0, "FACOAL180t2pp" => 0.0, "METSOXR1" => 0.0, "LIPOtex" => 0.0, "NTD11" => 0.0, "GLUNpp" => 0.0…)
 Dict("Zn2tex" => 0.00033498878813989854, "GUI1" => 0.0, "DXYLK" => 0.0, "CBL1tonex" => 0.0, "FE3DCITtonex" => 0.0, "FACOAL180t2pp" => 0.0, "METSOXR1" => 0.0, "LIPOtex" => 0.0, "NTD11" => 0.0, "GLUNpp" => 0.0…)
 Dict("Zn2tex" => 0.00033498878813989854, "GUI1" => 0.0, "DXYLK" => 0.0, "CBL1tonex" => 0.0, "FE3DCITtonex" => 0.0, "FACOAL180t2pp" => 0.0, "METSOXR1" => 0.0, "LIPOtex" => 0.0, "NTD11" => 0.0, "GLUNpp" => 0.0…)
 Dict("Zn2tex" => 0.00033498878813989854, "GUI1" => 0.0, "DXYLK" => 0.0, "CBL1tonex" => 0.0, "FE3DCITtonex" => 0.0

...but that might be slow for larger amounts of genes, and we would like to
add some special handling for knockouts where there is no feasible solution
(and the function returns `nothing`).

First, let's use COBREXA parallelization capabilities to make this faster. We
use Distributed package to run this over multiple processes:

In [9]:
using Distributed
#addprocs(8)  # only add processes if you are sure that you have sufficient resources available!

load our stuff on the small cluster

In [10]:
#@everywhere using COBREXA, GLPK # only necessary if you added the extra processes

`screen` function allows us to run many analyses on a model with parallel,
with many optimizations related for distributed processing (e.g., data are
only moved once).

In [11]:
knockout_fluxes = screen(
    model, # the model which we process
    args = tuple.(genes(model)[1:10]), # all argument lists for the analyses
    analysis = (model, gene) -> # the analysis function ("lambda") that we want to run on the cluster for each item from the argument list
        flux_balance_analysis_dict(model, GLPK.Optimizer, modifications = [knockout(gene)]),
    workers = workers(), # this gives it the list of worker nodes to use
)

10-element Vector{Dict{String, Float64}}:
 Dict("Zn2tex" => 0.00033498878813989854, "GUI1" => 0.0, "DXYLK" => 0.0, "CBL1tonex" => 0.0, "FE3DCITtonex" => 0.0, "FACOAL180t2pp" => 0.0, "METSOXR1" => 0.0, "LIPOtex" => 0.0, "NTD11" => 0.0, "GLUNpp" => 0.0…)
 Dict("Zn2tex" => 0.00033498878813989854, "GUI1" => 0.0, "DXYLK" => 0.0, "CBL1tonex" => 0.0, "FE3DCITtonex" => 0.0, "FACOAL180t2pp" => 0.0, "METSOXR1" => 0.0, "LIPOtex" => 0.0, "NTD11" => 0.0, "GLUNpp" => 0.0…)
 Dict("Zn2tex" => 0.00033498878813989854, "GUI1" => 0.0, "DXYLK" => 0.0, "CBL1tonex" => 0.0, "FE3DCITtonex" => 0.0, "FACOAL180t2pp" => 0.0, "METSOXR1" => 0.0, "LIPOtex" => 0.0, "NTD11" => 0.0, "GLUNpp" => 0.0…)
 Dict("Zn2tex" => 0.00033498878813989854, "GUI1" => 0.0, "DXYLK" => 0.0, "CBL1tonex" => 0.0, "FE3DCITtonex" => 0.0, "FACOAL180t2pp" => 0.0, "METSOXR1" => 0.0, "LIPOtex" => 0.0, "NTD11" => 0.0, "GLUNpp" => 0.0…)
 Dict("Zn2tex" => 0.00033498878813989854, "GUI1" => 0.0, "DXYLK" => 0.0, "CBL1tonex" => 0.0, "FE3DCITtonex" => 0.0

Exercise: Try processing more genes with and without the workers parameter to
see the speed-up.

Now that we see that it works, let's post-process the results a little, and
also add more genes:

In [12]:
knockout_fluxes = screen(
    model,
    args = tuple.(genes(model)[1:50]),
    analysis = (model, gene) -> begin
        res = flux_balance_analysis_dict(model, GLPK.Optimizer, modifications = [knockout(gene)])
        if !isnothing(res)
            gene => res["BIOMASS_Ec_iJO1366_core_53p95M"]
        else
            gene => 0.0
        end
    end,
    workers = workers(),
)

50-element Vector{Pair{String, Float64}}:
 "b1377" => 0.9823718127269752
 "b0241" => 0.9823718127269752
 "b0929" => 0.9823718127269752
 "b2215" => 0.9823718127269752
 "b0653" => 0.9823718127269752
 "b0655" => 0.9823718127269752
 "b0118" => 0.9823718127269752
 "b1276" => 0.9823718127269752
 "b4032" => 0.9823718127269772
 "b3359" => -1.3672895417877006e-17
         ⋮
 "b3737" => 0.4024773014907232
 "b3733" => 0.4024773014907232
 "b3734" => 0.4024773014907232
 "b3738" => 0.4024773014907232
 "b1009" => 0.9823718127269752
 "b1812" => -1.1020296920314708e-30
 "b0180" => 0.0
 "b3360" => -1.1020296920314708e-30
 "b3731" => 0.4024773014907232

After everything works, you can erase the limit to the first 50 genes and see
a complete result.

Let's create a CSV with a report, as always

In [13]:
Pkg.add(["DataFrames","CSV"])
using DataFrames, CSV

df = DataFrame(gene = first.(knockout_fluxes))
df.name = gene_name.(Ref(model), df.gene)
df.fluxes = last.(knockout_fluxes)
df

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


Row,gene,name,fluxes
,String,String,Float64
1,b1377,ompN,0.982372
2,b0241,phoE,0.982372
3,b0929,ompF,0.982372
4,b2215,ompC,0.982372
5,b0653,gltK,0.982372
6,b0655,gltI,0.982372
7,b0118,acnB,0.982372
8,b1276,acnA,0.982372
9,b4032,malG,0.982372


Typically we want to mark the genes that changed something. Let's mark the
genes that are required for growth as essential, and the ones that reduce the
growth somehow (but not fatally) as interesting.

In [14]:
best_result = maximum(last.(knockout_fluxes))
essential_threshold = 0.01 * best_result
df.essential = df.fluxes .<= essential_threshold
df.interesting = (df.fluxes .< best_result * 0.999) .&& .!df.essential
df

CSV.write("ko_report.csv", df)

"ko_report.csv"

## Doing the knockouts manually, the hard way

Now, let's have a look at how the knockouts are computed.

Each reaction has a Gene-Reaction Rule (GRR) that marks the genes required
for it to actually work in the organism. These are generally any Boolean
expressions, but in COBREXA we tend to store them in disjunctive normal form
(DNF, see https://en.wikipedia.org/wiki/Disjunctive_normal_form) which
closely corresponds to the biological meaning of gene units that form
interchangeable complexes. You can access them using the `grr` field in
Reaction structures:

In [15]:
model.reactions["RNDR2"].grr

2-element Vector{Vector{String}}:
 ["b2234", "b2235", "b2582"]
 ["b2234", "b2235", "b3781"]

Here, the reaction can be supported by either of the 2 possibilities (enzyme
complexes) where the first possibility is built from gene products of genes
`b2234`, `b2235`, and `b2582`; and as the second possibility it can also use
`b3781` instead of the `b2582`.

We may list all GRRs simply by iterating through the model reactions:

In [16]:
[rid => r.grr for (rid,r) in model.reactions]

2583-element Vector{Pair{String}}:
      "EX_cm_e" => nothing
     "EX_cmp_e" => nothing
     "EX_co2_e" => nothing
 "EX_cobalt2_e" => nothing
  "DM_4crsol_c" => nothing
   "DM_5drib_c" => nothing
  "DM_aacald_c" => nothing
    "DM_amob_c" => nothing
  "DM_mththf_c" => nothing
  "EX_colipa_e" => nothing
                ⋮
       "RNDR2b" => [["b0849", "b2675", "b2676"], ["b1064", "b2675", "b2676"], ["b1654", "b2675", "b2676"], ["b2675", "b2676", "b3610"]]
      "UREAtpp" => [["b3927"]]
        "RNDR3" => [["b2234", "b2235", "b2582"], ["b2234", "b2235", "b3781"]]
       "RNDR3b" => [["b0849", "b2675", "b2676"], ["b1064", "b2675", "b2676"], ["b1654", "b2675", "b2676"], ["b2675", "b2676", "b3610"]]
        "RNDR4" => [["b2234", "b2235", "b2582"], ["b2234", "b2235", "b3781"]]
       "RNDR4b" => [["b0849", "b2675", "b2676"], ["b1064", "b2675", "b2676"], ["b1654", "b2675", "b2676"], ["b2675", "b2676", "b3610"]]
      "RNTR1c2" => [["b0684", "b3924", "b4237", "b4238"], ["b0684", "b4238"], ["b2

It is often interesting to ask which reactions may depend on which gene, we
can make a convenience function for that:

In [17]:
reactions_of_gene(model, gene) =
  [rid for (rid,r) in model.reactions if !isnothing(r.grr) && any(complex -> any(contains(gene), complex), r.grr)]

reactions_of_gene(model, "b1064")

7-element Vector{String}:
 "ASR"
 "GRXR"
 "PAPSR2"
 "RNDR1b"
 "RNDR2b"
 "RNDR3b"
 "RNDR4b"

Using the vector notation is quite convenient for creating lists that allow
us to get an overview of the situation:

In [18]:
gene_name.(Ref(model), genes(model)) .=> reactions_of_gene.(Ref(model), genes(model))

1367-element Vector{Pair{String, Vector{String}}}:
 "ompN" => ["12PPDRtex", "12PPDStex", "23CAMPtex", "23CCMPtex", "23CGMPtex", "23CUMPtex", "23DAPPAtex", "26DAHtex", "34dhpactex", "3AMPtex"  …  "RIBtex", "UDPGLCURtex", "XYLUtex", "UDPGtex", "XYLtex", "Zn2tex", "RMNtex", "UMPtex", "URAtex", "UREAtex"]
 "phoE" => ["12PPDRtex", "12PPDStex", "23CAMPtex", "23CCMPtex", "23CGMPtex", "23CUMPtex", "23DAPPAtex", "26DAHtex", "34dhpactex", "3AMPtex"  …  "RIBtex", "UDPGLCURtex", "XYLUtex", "UDPGtex", "XYLtex", "Zn2tex", "RMNtex", "UMPtex", "URAtex", "UREAtex"]
 "ompF" => ["12PPDRtex", "12PPDStex", "23CAMPtex", "23CCMPtex", "23CGMPtex", "23CUMPtex", "23DAPPAtex", "26DAHtex", "34dhpactex", "3AMPtex"  …  "RIBtex", "UDPGLCURtex", "XYLUtex", "UDPGtex", "XYLtex", "Zn2tex", "RMNtex", "UMPtex", "URAtex", "UREAtex"]
 "ompC" => ["12PPDRtex", "12PPDStex", "23CAMPtex", "23CCMPtex", "23CGMPtex", "23CUMPtex", "23DAPPAtex", "26DAHtex", "34dhpactex", "3AMPtex"  …  "RIBtex", "UDPGLCURtex", "XYLUtex", "UDPGtex", "X

Now, given a set of genes that we want to knock out, we can manually find if
a given reaction will still work or not. Let's try on RNDR1:

In [19]:
grr = model.reactions["RNDR1"].grr

ko_genes = ["b2234"]

1-element Vector{String}:
 "b2234"

We can transform the `grr` to a form where it says which genes are present
and which genes are not:

In [20]:
grr_available = map(c -> map(!in(ko_genes), c), grr)

2-element Vector{Vector{Bool}}:
 [0, 1, 1]
 [0, 1, 1]

To determine if the reaction _can_ work, at least one ("any") of the
complexes must have all gene products available:

In [21]:
any(all, grr_available)

false

Since `b2234` is essential for RNDR1 (it needs to be present in all complexes
that may run the reaction), the reaction is effectively disabled by knocking
out `b2234`.

What if we knock out `b2582`?

In [22]:
ko_genes = ["b2582"]
grr_available = map(c -> map(!in(ko_genes), c), grr)

any(all, grr_available)

true

...the reaction may still work with just `b2582` knocked out.

Anyway, if we knock out multiple genes, the reaction will cease to work again:

In [23]:
ko_genes = ["b2582", "b3781"]
grr_available = map(c -> map(!in(ko_genes), c), grr)
any(all, grr_available)

false

We can formalize the knockout evaluation in a function

In [24]:
function is_reaction_knocked_out(model, reaction, ko_genes)
    grr = model.reactions[reaction].grr
    if isnothing(grr)
        return false # reactions without a gene-reaction rule happen spontaneously and cannot be knocked out
    end
    grr_available = map(c -> map(!in(ko_genes), c), grr)
    !any(all, grr_available)
end

is_reaction_knocked_out (generic function with 1 method)

Now, we can manually modify the model to disable the reactions that would be
knocked out by a certain gene combination:

In [25]:
ko_genes = ["b2582", "b3781"]
for (rid, r) = model.reactions
    if is_reaction_knocked_out(model, rid, ko_genes)
        r.lb = r.ub = 0.0
    end
end

Does the model still grow?

In [26]:
sol = flux_balance_analysis_dict(model, GLPK.Optimizer)
sol["BIOMASS_Ec_iJO1366_core_53p95M"]

0.9823718127269752

...which seems like the combination of the 2 genes was not essential at all.

## How well do the in-silico knockouts compare to real measurements?

Since there are existing measurements of what happens with real E. Coli after
knockouts, we can look at our results as predictions, and compare them to the
ground truth with the usual statistical means.

First, let's read the experimentally verified "lethal" knockout genes from
the supplied JSON data file

In [27]:
using JSON

This is the list of lethal gene knockouts:

In [28]:
ex_lethal = JSON.parsefile("data/m9_invivo_lethals.json")

238-element Vector{Any}:
 "b0004"
 "b0003"
 "b0002"
 "b2925"
 "b0414"
 "b0415"
 "b0417"
 "b1637"
 "b3623"
 "b2515"
 ⋮
 "b1281"
 "b2320"
 "b1288"
 "b2323"
 "b3648"
 "b3640"
 "b3642"
 "b2563"
 "b4006"

Let's convert it to vectorized representation for the dataframe:

In [29]:
df.invivo_essential = in.(df.gene, Ref(ex_lethal))

df.insilico_essential = df.essential #rename for clarity

50-element BitVector:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 ⋮
 0
 0
 0
 0
 0
 1
 1
 1
 0

Now we can count true&false positives&negatives:

In [30]:
TP = count(df.insilico_essential .& df.invivo_essential);
TN = count(.!df.insilico_essential .& .!df.invivo_essential);
FP = count(.!df.insilico_essential .& df.invivo_essential);
FN = count(df.insilico_essential .& .!df.invivo_essential);

Let's arrange these in a standard confusion matrix:

In [31]:
confusion_mtx = [
    TP FN
    FP TN
]

2×2 Matrix{Int64}:
 4   2
 4  40

This allows us to compute some useful metrics about the predictions:

In [32]:
(
    sensitivity = TP / (TP + FN),
    specificity = TN / (TN + FP),
    accuracy = (TP + TN) / sum(confusion_mtx),
    f1_score = (2 * TP) / (2 * TP + FN + FP),
)

(sensitivity = 0.6666666666666666, specificity = 0.9090909090909091, accuracy = 0.88, f1_score = 0.5714285714285714)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*